# DRAFT
# Accuracy assessment of classification results

This notebook is set up for generating a confusion matrix for a binary classification.  It would require editing to creating a confusion matrix for multi-class classifications.

Inputs will be:

1. `predicted.tif` : a binary classification of crop/no-crop output by the ML script.

2. `validation.shp` : a shapefile containing crop/no-crop points to serve as the "ground-truth" dataset

3. `simplified_AEZ.shp` : a shapefile used to limit the ground truth points to the region where the model has classified crop/non-crop

Output will be:
1. A `confusion error matrix` containing Overall, Producer's, and User's accuracy, along with the F1 score for each class

In [1]:
import rasterio
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.metrics import f1_score

## Analysis Parameters

In [2]:
pred_tif = 'results/predicted.tif'
grd_truth = 'data/training_validation/GFSAD2015/ground_truth_example.shp'
aez = 'data/Southern.shp'

### Load the datasets

`Ground truth points`

In [3]:
#ground truth shapefile
ground_truth = gpd.read_file(grd_truth).to_crs('EPSG:6933')

In [4]:
# rename the class column to 'actual'
ground_truth = ground_truth.rename(columns={'class':'Actual'})
ground_truth.head()

,Actual,geometry
0,0,POINT (2348960.082 2523843.580)
1,0,POINT (553136.870 2547155.762)
2,0,POINT (2278070.581 2571794.107)
3,0,POINT (608920.945 2660250.463)
4,0,POINT (2354922.751 2669991.398)


Clip ground_truth data points to the simplified AEZ

In [5]:
#open shapefile
aez=gpd.read_file(aez_region).to_crs('EPSG:6933')
# clip points to region
ground_truth = gpd.overlay(ground_truth,aez,how='intersection')

`Raster of predicted classes`

In [5]:
prediction = rasterio.open(pred_tif)

### Extract a list of coordinate values

In [7]:
coords = [(x,y) for x, y in zip(ground_truth.geometry.x, ground_truth.geometry.y)]

### Sample the prediction raster at the ground truth coordinates

In [8]:
# Sample the raster at every point location and store values in DataFrame
ground_truth['Prediction'] = [int(x[0]) for x in prediction.sample(coords)]
ground_truth.head()

,Actual,ID,CODE,COUNTRY,geometry,Prediction
0,1,21,BOT,Southern,POINT (1907192.607 -4168598.608),1
1,1,21,BOT,Southern,POINT (1947076.234 -4115733.446),0
2,0,21,BOT,Southern,POINT (2060234.429 -4070146.502),0
3,0,21,BOT,Southern,POINT (2482258.847 -4060779.525),0
4,0,21,BOT,Southern,POINT (2102370.619 -4043333.246),0


---

## Create a confusion matrix

In [9]:
confusion_matrix = pd.crosstab(ground_truth['Actual'],
                               ground_truth['Prediction'],
                               rownames=['Actual'],
                               colnames=['Prediction'],
                               margins=True)

confusion_matrix

Prediction,0,1,All
Actual,,,
0,236,0,236
1,50,6,56
All,286,6,292


### Calculate User's and Producer's Accuracy

`User's Accuracy`

In [10]:
confusion_matrix["User's"] = [confusion_matrix.loc[0, 0] / confusion_matrix.loc[0, 'All'] * 100,
                              confusion_matrix.loc[1, 1] / confusion_matrix.loc[1, 'All'] * 100,
                              np.nan]

`Producer's Accuracy`

In [11]:
producers_accuracy = pd.Series([confusion_matrix[0][0] / confusion_matrix[0]['All'] * 100,
                                confusion_matrix[1][1] / confusion_matrix[1]['All'] * 100]
                         ).rename("Producer's")

confusion_matrix = confusion_matrix.append(producers_accuracy)

`Overall Accuracy`

In [12]:
confusion_matrix.loc["Producer's", "User's"] = (confusion_matrix.loc[0, 0] + 
                                                confusion_matrix.loc[1, 1]) / confusion_matrix.loc['All', 'All'] * 100

`F1 Score`

The F1 score is the harmonic mean of the precision and recall, where an F1 score reaches its best value at 1 (perfect precision and recall), and is calculated as:

$$
\begin{aligned}
\text{Fscore} = 2 \times \frac{\text{UA} \times \text{PA}}{\text{UA} + \text{PA}}.
\end{aligned}
$$

Where UA = Users Accuracy, and PA = Producer's Accuracy

In [13]:
fscore = pd.Series([(2*(confusion_matrix.loc[0, "User's"]*confusion_matrix.loc["Producer's", 0]) / (confusion_matrix.loc[0, "User's"]+confusion_matrix.loc["Producer's", 0])) / 100,
                    f1_score(ground_truth['Actual'], ground_truth['Prediction'], average='binary')]
                         ).rename("F-score")

confusion_matrix = confusion_matrix.append(fscore)

### Tidy Confusion Matrix

* Limit decimal places,
* Add readable class names
* Remove non-sensical values 

In [15]:
# round numbers
confusion_matrix = confusion_matrix.round(decimals=2)

In [16]:
# rename booleans to class names
confusion_matrix = confusion_matrix.rename(columns={0:'Non-crop', 1:'Crop', 'All':'Total'},
                                            index={0:'Non-crop', 1:'Crop', 'All':'Total'})

In [17]:
#remove the nonsensical values in the table
confusion_matrix.loc['Total', "User's"] = '--'
confusion_matrix.loc["Producer's", 'Total'] = '--'
confusion_matrix.loc["F-score", 'Total'] = '--'
confusion_matrix.loc["F-score", "User's"] = '--'

In [18]:
confusion_matrix

Prediction,Non-crop,Crop,Total,User's
Actual,,,,
Non-crop,236.00,0.00,236,100
Crop,50.00,6.00,56,10.71
Total,286.00,6.00,292,--
Producer's,82.52,100.00,--,82.88
F-score,0.90,0.19,--,--


### Export csv

In [19]:
# confusion_matrix.to_csv('results/confusion_matrix.csv')